In [43]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [44]:
df_nopass = pd.concat([pd.read_csv("results/out_nopass/faults_memory.csv"),
                       pd.read_csv("results/out_nopass/faults_registers.csv")])

df_eddi_cfcss = pd.concat([pd.read_csv("results/out_eddi_cfcss/faults_memory.csv"),
                           pd.read_csv("results/out_eddi_cfcss/faults_registers.csv")])

df_eddi_rasm_intra = pd.concat([pd.read_csv("results/out_eddi_rasm_intra/faults_memory.csv"),
                                pd.read_csv("results/out_eddi_rasm_intra/faults_registers.csv")])

df_fdsc_cfcss = pd.concat([pd.read_csv("results/out_fdsc_cfcss/faults_memory.csv"),
                           pd.read_csv("results/out_fdsc_cfcss/faults_registers.csv")])

df_fdsc_rasm_intra = pd.concat([pd.read_csv("results/out_fdsc_rasm_intra/faults_memory.csv"),
                                pd.read_csv("results/out_fdsc_rasm_intra/faults_registers.csv")])

df_fdsc_rasm = pd.concat([pd.read_csv("results/out_fdsc_rasm/faults_memory.csv"),
                          pd.read_csv("results/out_fdsc_rasm/faults_registers.csv")])

dataframes = {"df_eddi_cfcss": df_eddi_cfcss,
              "df_eddi_rasm_intra": df_eddi_rasm_intra,
              "df_fdsc_cfcss": df_fdsc_cfcss,
              "df_fdsc_rasm_intra": df_fdsc_rasm_intra,
              "df_fdsc_rasm": df_fdsc_rasm}

dataframes["df_nopass"] = df_nopass

In [45]:
ind = np.arange(6)
width = .25

df_new = pd.DataFrame()

for key in dataframes:
    df = dataframes[key]
    sz_tot = df.shape[0]
    df_errcodes = df.groupby('code')
    sz_df_errcodes = df_errcodes.size()
    print(sz_df_errcodes)
    df_new[key] = sz_df_errcodes

code
-3      241
-2        7
-1      453
 0    27543
 1      969
 2      787
dtype: int64
code
-3      174
-2        7
-1      879
 0    27011
 1     1183
 2      746
dtype: int64
code
-3      287
-2       11
-1      544
 0    27808
 1      498
 2      852
dtype: int64
code
-3      224
-2       20
-1      985
 0    27294
 1      696
 2      781
dtype: int64
code
-3      770
-2       39
-1      929
 0    28262
dtype: int64


In [46]:
df_new

,df_eddi_cfcss,df_eddi_rasm_intra,df_fdsc_cfcss,df_fdsc_rasm_intra,df_nopass
code,,,,,
-3,241,174,287,224,770.0
-2,7,7,11,20,39.0
-1,453,879,544,985,929.0
0,27543,27011,27808,27294,28262.0
1,969,1183,498,696,NaN
2,787,746,852,781,NaN


In [47]:
df_new / 300

,df_eddi_cfcss,df_eddi_rasm_intra,df_fdsc_cfcss,df_fdsc_rasm_intra,df_nopass
code,,,,,
-3,0.803333,0.580000,0.956667,0.746667,2.566667
-2,0.023333,0.023333,0.036667,0.066667,0.130000
-1,1.510000,2.930000,1.813333,3.283333,3.096667
0,91.810000,90.036667,92.693333,90.980000,94.206667
1,3.230000,3.943333,1.660000,2.320000,NaN
2,2.623333,2.486667,2.840000,2.603333,NaN


In [51]:
df_new.drop(0) / df_new.drop(0).sum() * 100

,df_eddi_cfcss,df_eddi_rasm_intra,df_fdsc_cfcss,df_fdsc_rasm_intra,df_nopass
code,,,,,
-3,9.808710,5.821345,13.093066,8.277901,44.303797
-2,0.284900,0.234192,0.501825,0.739098,2.243959
-1,18.437118,29.407829,24.817518,36.400591,53.452244
1,39.438339,39.578454,22.718978,25.720621,NaN
2,32.030932,24.958180,38.868613,28.861789,NaN
